In [4]:
from pydataset import data

mpg = data('mpg')
data('mpg', show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




On average, which manufacturer has the best miles per gallon?

How many different manufacturers are there?

How many different models are there?

Do automatic or manual cars have better miles per gallon?

# 2. Joining and Merging

Copy the users and roles dataframes from the examples above.

What do you think a right join would look like?

An outer join?

What happens if you drop the foreign keys from the dataframes and try to merge them?

# 3. Getting data from SQL databases

Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url formatted like in the examples in this lesson.

Use your function to obtain a connection to the employees database.